In [581]:
import pandas as pd
# Asegurar que todas las columnas se muestren
pd.set_option('display.max_rows', None)  # Mostrar todas las filas
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [582]:
null_values = [" "]

# Leer el archivo con estos valores tratados como nulos
data = pd.read_csv("../01_Data/data.csv", encoding="latin1", na_values=null_values)

data = data[data['RESFIN'] == 1]
data = data.drop(columns = [
    'ANIO','OMICAP200','CCDD','CCPP','CCDI','CONGLOMERADO',
    'NSELUA','UA','RESFIN','P102_1','P105_N','P203','CODIGO',
    'P223A','P214','P221_1','P221_2','P215','P204_COD'])

In [583]:
for col in data.columns:
    if col.endswith('_ENT'):
        base_name = col[:-4]  # Eliminar el sufijo '_ENT'
        decimal_col = f"{base_name}_DEC"  # Buscar la columna con sufijo '_DEC'

        if decimal_col in data.columns:
            # Extraer partes enteras y decimales como strings
            ent_part = data[col].astype(str).str.split(".").str[0]  # Solo parte entera
            dec_part = data[decimal_col].astype(str).str.split(".").str[0]  # Solo parte decimal

            # Crear nueva columna combinada (entera + decimal ajustada)
            data[base_name] = pd.to_numeric(ent_part + "." + dec_part, errors='coerce')  # Parte entera como número

            # Opcional: Eliminar las columnas originales _ENT y _DEC
            data.drop([col, decimal_col], axis=1, inplace=True)

In [584]:
data.columns, data.shape

(Index(['NOMBREDD', 'NOMBREPV', 'NOMBREDI', 'REGION', 'DOMINIO', 'FACTOR',
        'P101A', 'P102_2', 'P204_NOM', 'P204_TIPO', 'P205_N', 'P205_TOT',
        'P206_INI_MES', 'P206_INI_ANIO', 'P207_FIN_MES', 'P207_FIN_ANIO',
        'P208', 'P209_MES', 'P209_ANIO', 'P210_SUP_1', 'P210_SUP_2', 'P211_1',
        'P211_2', 'P211_3', 'P211_4', 'P211_5', 'P211_6', 'P211_7', 'P211_8',
        'P212', 'P213', 'P217_SUP_1', 'P217_SUP_2', 'P217_SUP_ha', 'P218',
        'P218A', 'P218B', 'P219_CANT_1', 'P219_CANT_2', 'P219_UM',
        'P219_UM_COD', 'P219_EQUIV_KG', 'P220_1_CANT_1', 'P220_1_CANT_2',
        'P220_1_PREC_1', 'P220_1_PREC_2', 'P220_1_VAL', 'P220_1_PRE_KG',
        'P220_2_PREC_1', 'P220_2_PREC_2', 'P220_2_VAL', 'P220_2_PRE_KG',
        'P220_3A_PREC_1', 'P220_3A_PREC_2', 'P220_3A_VAL', 'P220_3A_PRE_KG',
        'P220_3B_PREC_1', 'P220_3B_PREC_2', 'P220_3B_VAL', 'P220_3B_PRE_KG',
        'P222_1', 'P222_2', 'P222_3', 'P222_4', 'P222_5', 'P222_6', 'P222_7',
        'P223_1', 'P223_2'

In [585]:
# Total de filas en el DataFrame
total_filas = len(data)

# Calcular el porcentaje de valores nulos por columna
nulos_por_columna = data.isnull().sum()
porcentaje_nulos = (nulos_por_columna / total_filas) * 100

# Filtrar columnas con valores nulos
columnas_con_nulos = porcentaje_nulos[porcentaje_nulos > 0]

# Mostrar el porcentaje de nulos por columna
print(columnas_con_nulos.sort_values(ascending=False))

P101A             100.000000
P220_3A_PREC_2    100.000000
P220_3A_PREC_1    100.000000
P220_3A_VAL       100.000000
P220_3B_VAL       100.000000
P220_3B           100.000000
P220_3A           100.000000
P220_3B_PRE_KG    100.000000
P220_3B_PREC_1    100.000000
P220_3B_PREC_2    100.000000
P220_3A_PRE_KG    100.000000
P218A              86.508726
P218B              86.508726
P218               66.066339
P223B_2            65.323978
P223B_1            65.323978
P223B_5            65.323978
P223B_6            65.323978
P223B_4            65.323978
P223B_3            65.323978
P223B_7            65.323978
P223B_8            65.323978
P220_3             55.961140
P213               53.248303
P220_1_VAL         51.648060
P220_1_PREC_1      51.648060
P220_1_PRE_KG      51.648060
P223_4             51.648060
P222_3             51.648060
P223_3             51.648060
P223_2             51.648060
P223_1             51.648060
P222_7             51.648060
P222_6             51.648060
P222_5        

In [586]:
# Eliminar columnas con al menos 70% de valores nulos
data = data.loc[:, porcentaje_nulos <= 70]

In [587]:
# Variables predictoras y objetivo
predictors = ['P217_SUP_1', 'P217_SUP_2', 'P217_SUP_ha']
target = 'P218'
group_column = 'P204_NOM'  # Columna para segmentar por cultivo

# Contadores para resultados
imputed_count = 0
deleted_rows_count = 0

# Iterar por cada tipo de cultivo
for cultivo in data[group_column].dropna().unique():
    # Filtrar datos para el cultivo actual
    cultivo_data = data[data[group_column] == cultivo]
    
    # Separar datos para entrenamiento y predicción
    train_data = cultivo_data.dropna(subset=predictors + [target])
    test_data = cultivo_data[cultivo_data[target].isnull()]
    
    if len(train_data) > 10:  # Entrenar el modelo solo si hay suficientes datos
        # Entrenar modelo de regresión
        X_train = train_data[predictors]
        y_train = train_data[target]
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predecir valores para los datos faltantes
        X_test = test_data[predictors].dropna()
        if not X_test.empty:
            predicted_values = model.predict(X_test)
            predicted_values = np.round(predicted_values).astype(int)  # Redondear al entero más cercano
            
            # Imputar los valores predichos
            data.loc[X_test.index, target] = predicted_values
            imputed_count += len(predicted_values)
            print(f"Imputados {len(predicted_values)} valores en '{target}' para cultivo '{cultivo}'.")
    else:
        # Si no hay suficientes datos, imputar con la mediana o eliminar
        median_value = train_data[target].median()
        if not np.isnan(median_value):
            missing_before = data[target].isnull().sum()
            data.loc[(data[group_column] == cultivo) & (data[target].isnull()), target] = median_value
            missing_after = data[target].isnull().sum()
            imputed_count += missing_before - missing_after
        else:
            # Eliminar filas si no se puede calcular la mediana
            rows_to_delete = data[(data[group_column] == cultivo) & (data[target].isnull())].index
            deleted_rows_count += len(rows_to_delete)
            data = data.drop(rows_to_delete)

print(f"Valores imputados: {imputed_count}")
print(f"Filas eliminadas: {deleted_rows_count}")

Valores imputados: 252
Filas eliminadas: 69520


In [588]:
# Columnas a procesar
columns_to_process = [
    'P220_1_PREC_1', 'P220_1_PRE_KG', 'P220_1_VAL', 'P220_1_PREC_2', 
    'P220_2_PRE_KG', 'P220_2_VAL', 'P220_2_PREC_1', 'P220_2_PREC_2',
    'P220_6'
]
group_column = 'P204_NOM'  # Columna para segmentar por tipo de cultivo

# Contadores para resultados
imputed_count = 0
deleted_rows_count = 0

# Iterar por cada columna a procesar
for col in columns_to_process:
    # Iterar por cada tipo de cultivo
    for cultivo in data[group_column].dropna().unique():
        # Filtrar datos para el cultivo actual
        cultivo_data = data[data[group_column] == cultivo]
        
        # Validar que haya datos suficientes para calcular la mediana
        if not cultivo_data[col].dropna().empty:  # Verificar que el subconjunto no esté vacío
            # Calcular la mediana de la columna para este cultivo
            median_value = cultivo_data[col].median()
            
            # Reemplazar los valores nulos con la mediana
            missing_before = data[col].isnull().sum()
            data.loc[(data[group_column] == cultivo) & (data[col].isnull()), col] = median_value
            missing_after = data[col].isnull().sum()
            imputed_count += missing_before - missing_after
        else:
            # Si no hay datos para calcular la mediana, eliminar las filas
            rows_to_delete = data[(data[group_column] == cultivo) & (data[col].isnull())].index
            deleted_rows_count += len(rows_to_delete)
            data = data.drop(rows_to_delete)

print(f"Valores imputados: {imputed_count}")
print(f"Filas eliminadas: {deleted_rows_count}")

Valores imputados: 99166
Filas eliminadas: 5704


In [590]:
# Eliminar filas donde P209_MES o P209_ANIO tienen valores nulos
data = data.dropna(subset=['P209_MES', 'P209_ANIO'])

# Rellenar los valores nulos con 0 en las columnas que comienzan con P222 y P223
data[[col for col in data.columns if col.startswith('P222') or col.startswith('P223')]] = \
    data[[col for col in data.columns if col.startswith('P222') or col.startswith('P223')]].fillna(0)

# Reemplazar nulos en P220_3_ENT y P220_3_DEC con 0
data[['P220_3']] = data[['P220_3']].fillna(0)

# Reemplazar nulos en P213 con la moda
moda_p213 = data['P213'].mode()[0]  # Calcular la moda
data['P213'] = data['P213'].fillna(moda_p213)

# Imputar valores nulos con la mediana
mediana = data['P220_5'].median()  # Calcular la mediana
data['P220_5'] = data['P220_5'].fillna(mediana)  # Reemplazar nulos con la mediana

In [591]:
data.isna().sum().sum(), data.shape

(np.int64(0), (30094, 79))

In [592]:
data.to_csv("../01_Data/clean_data.csv",index=False)